In [ ]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import utils.behavioral_utils as behavioral_utils
import utils.information_utils as information_utils
import utils.visualization_utils as visualization_utils
import utils.glm_utils as glm_utils
from matplotlib import pyplot as plt
import utils.spike_utils as spike_utils

In [ ]:
OUTPUT_DIR = "/data/patrick_res/glm"

session = 20180802
separate_res = pd.read_pickle(os.path.join(OUTPUT_DIR, f"{session}_glm_feature_rpe_separate.pickle"))
interaction_res = pd.read_pickle(os.path.join(OUTPUT_DIR, f"{session}_glm_feature_rpe_interaction.pickle"))
shuffled_res = pd.read_pickle(os.path.join(OUTPUT_DIR, f"{session}_glm_feature_rpe_shuffles.pickle"))

# interaction_res = pd.read_pickle(os.path.join(OUTPUT_DIR, f"{session}_glm_card_interaction.pickle"))
# shuffled_res = pd.read_pickle(os.path.join(OUTPUT_DIR, f"{session}_glm_card_shuffles.pickle"))

In [ ]:
stats = glm_utils.calculate_sig_stats(shuffled_res, 0.05, 15)

In [ ]:
feedback_bins = np.arange(1.3, 2.8, 0.1)
choice_bins = np.arange(0, 1.3, 0.1)

In [ ]:
# units_sig = glm_utils.identify_significant_units(interaction_res, shuffled_res, feedback_bins)

choice_bins = np.arange(0, 1.3, 0.1)
units_sig = glm_utils.identify_significant_units(interaction_res, shuffled_res, choice_bins)
stats = glm_utils.calculate_sig_stats(shuffled_res, 0.05, 13)

In [ ]:
len(units_sig[units_sig.IsSig])

In [ ]:
units_sig = units_sig[units_sig.IsSig]

In [ ]:
pos = spike_utils.get_unit_positions_per_sess(session)
pos = spike_utils.get_manual_structure(pos)

In [ ]:
for unit in units_sig.UnitID.unique():
    fig, ax = plt.subplots()
    unit_pos = pos[pos.UnitID == unit].manual_structure.unique()[0]
    # unit_separate = separate_res[separate_res.UnitID == unit].score
    unit_interaction = interaction_res[interaction_res.UnitID == unit].score
    sig_bound = stats[stats.UnitID == unit].sig_bound

    time_bins = interaction_res[interaction_res.UnitID == unit].TimeBins - 1.3

    # ax.plot(time_bins, unit_separate, label="Separate Score")
    ax.plot(time_bins, unit_interaction, label="Interaction Score")
    ax.plot(time_bins, sig_bound, label="Significance Bound")

    ax.set_title(f"Unit {unit} ({unit_pos})")
    ax.legend()
    ax.axvspan(-0.8, 0, alpha=0.3, color='gray')
    ax.axvline(0.098, alpha=0.3, color='gray', linestyle='dashed')
    ax.set_xlabel("Time Relative to Feedback (s)")
    ax.set_ylabel("Fraction of Deviance Explained")

### Assess as a population, how many neurons fit this criteria

In [ ]:
SESSIONS_PATH = "/data/patrick_res/multi_sess/valid_sessions_rpe.pickle"
sessions = pd.read_pickle(SESSIONS_PATH)

In [ ]:
def find_sig_units_per_session(row, time_bins, card=False):
    session = row.session_name
    if card: 
        interaction_res = pd.read_pickle(os.path.join(OUTPUT_DIR, f"{session}_glm_card_interaction.pickle"))
        shuffled_res = pd.read_pickle(os.path.join(OUTPUT_DIR, f"{session}_glm_card_shuffles.pickle"))
    else: 
        interaction_res = pd.read_pickle(os.path.join(OUTPUT_DIR, f"{session}_glm_feature_rpe_interaction.pickle"))
        shuffled_res = pd.read_pickle(os.path.join(OUTPUT_DIR, f"{session}_glm_feature_rpe_shuffles.pickle"))        
    units_sig = glm_utils.identify_significant_units(interaction_res, shuffled_res, time_bins)
    sig_units = units_sig[units_sig.IsSig]
    pos = spike_utils.get_unit_positions_per_sess(session)
    pos = pos.fillna("unknown")
    pos = spike_utils.get_manual_structure(pos)
    filtered_pos = pos[pos.UnitID.isin(sig_units.UnitID)]
    return filtered_pos

In [ ]:
choice_card_units = pd.concat(sessions.apply(lambda x: find_sig_units_per_session(x, choice_bins, card=True), axis=1).values)
feedback_card_units = pd.concat(sessions.apply(lambda x: find_sig_units_per_session(x, feedback_bins, card=True), axis=1).values)
choice_rpe_units = pd.concat(sessions.apply(lambda x: find_sig_units_per_session(x, choice_bins, card=False), axis=1).values)
feedback_rpe_units = pd.concat(sessions.apply(lambda x: find_sig_units_per_session(x, feedback_bins, card=False), axis=1).values)


In [ ]:
print(len(choice_card_units))
print(len(feedback_card_units))
print(len(choice_rpe_units))
print(len(feedback_rpe_units))

how many units are shared in rpe interaction selective pre feedback and postfeedback 

In [ ]:
shared_rpe_units = choice_rpe_units[choice_rpe_units.PseudoUnitID.isin(feedback_rpe_units.PseudoUnitID)]

In [ ]:
len(shared_rpe_units)

In [ ]:
card_rpe_fb_units = feedback_card_units[feedback_card_units.PseudoUnitID.isin(feedback_rpe_units.PseudoUnitID)]

In [ ]:
len(card_rpe_fb_units)

In [ ]:
card_rpe_choice_units = choice_card_units[choice_card_units.PseudoUnitID.isin(choice_rpe_units.PseudoUnitID)]

In [ ]:
len(card_rpe_choice_units)